In [1]:
!pip install accelerate gradio transformers inflect huggingface sentencepiece torch fastcore torchvision hanziconv einops pycantonese OpenCC==1.1.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 46.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.3/778.3 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 99.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 124.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.6 MB/s eta 0:00:00:00:0100:

In [2]:
# auth
!pip install -U -q "huggingface_hub[cli]"

In [3]:
# enter huggingface token
from huggingface_hub import login
login()

In [1]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
from fastcore.all import *
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from SCQuestionAnswer import SCQuestionAnswer, CreateLyricsPrompt, tone_mappings
from utils import generate_lyrics

In [2]:
# load model
print("\u27a4 Loading tokenizer...")
base_model_name = "Mobut-songcomposer/Songcomposer-tone-specific"
tokz = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)

# with init_empty_weights():
model = None
if not Path("Songcomposer-tone-specific").exists():
    print("\u27a4 Loading fine-tuned model from remote...")
    model = AutoModelForCausalLM.from_pretrained(base_model_name, trust_remote_code=True, device_map="auto")
else:
    print("\u27a4 Loading fine-tuned model locally...")
    model = AutoModelForCausalLM.from_pretrained("./Songcomposer-tone-specific", trust_remote_code=True, device_map="auto")
print("\u2713 Done!")

➤ Loading tokenizer...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


➤ Loading fine-tuned model from remote...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Set max length to 2048


/opt/conda/lib/python3.10/site-packages/accelerate/utils/modeling.py:1381: UserWarning: Current model requires 704648448 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


✓ Done!


In [4]:
# create the gradio interface
demo = gr.Interface(
    fn=lambda topic, tones: generate_lyrics(tokz, model, topic, tones),
    inputs=[
        gr.Textbox(label="主題 (Topic)", lines=1),
        gr.Textbox(label="聲調 (Tone contours)", lines=1)
    ],
    outputs=[
        gr.Textbox(label="歌詞 - 1 (Generated lyrics - 1)", lines=1),
        gr.Textbox(label="歌詞 - 2 (Generated lyrics - 2)", lines=1),
        gr.Textbox(label="歌詞 - 3 (Generated lyrics - 3)", lines=1)
    ],
    examples = [
        ["內心的孤寂", "113316"],
        ["矛盾和焦慮", "663642"]
    ],
    title="廣東話歌詞生成器 (Cantonese Lyrics Generator) - Demo",
    description="這是一個廣東話歌詞生成器，輸入主題和聲調，即可就已提供聲調生成一段相對應的歌詞。 (This is a Cantonese lyrics generator. Input a topic and tone contours, and it will generate corresponding lyrics based on the provided tone contours.)"
)
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://e316c6ea716dca2fd2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


tone_list=[1, 1, 3, 3, 1, 6]
------attempt 0------

一声再见不易
------attempt 1------
